Breast: 
Credit: 
Leeter：
Spam: 

### 配置文件验证

In [1]:
from omegaconf import OmegaConf
hparams = OmegaConf.load('config_Spam.yaml') # 读取配置文件

In [3]:
hparams.max_epochs

In [ ]:
hparams.pro_type_file is None

In [ ]:
from modelModule.model5 import VAE5
import torch
model = VAE5(dim=16, pro_types=pro_types)

### 得到数据集相关属性

In [3]:
import pickle
import pandas as pd
import numpy as np
import torch
# data_pd = pd.read_csv('Letter_train.csv')
# col_names = list(data_pd)
# df[col_name].value_counts()
# pro_types = [('discrete',16) for _ in col_names]
# with open('Letter_pro_type.pkl', 'wb') as f:
#     pickle.dump(pro_types, f)

### dataloader验证

In [1]:
import torch
from dataModule.dataset1 import FlatDataset, ValidDataset
from torch.utils.data import DataLoader

In [2]:
train_dataset = FlatDataset('/root/autodl-tmp/VAE_pytorchLightning/Spam/Spam_without_label.csv', 
                    pro_type_file='Spam/Spam_pro_type.pkl',
                    replace_dict_file='Spam/Spam数值置换表.pkl')


In [2]:
ValidDataset = ValidDataset(miss_file='Spam/Spam模拟缺失_0_3.csv', 
                            complete_file='/root/autodl-tmp/VAE_pytorchLightning/Spam/Spam_without_label.csv',
                            pro_type_file='Spam/Spam_pro_type.pkl',
                            replace_dict_file='Spam/Spam数值置换表.pkl')

In [3]:
valid_dataloader = DataLoader(ValidDataset, 
                             batch_size=10, 
                             collate_fn=ValidDataset.collater, 
                             shuffle=False,
                             num_workers=2)
val_batch = None
for i in valid_dataloader:
    val_batch = i
    break

In [6]:
val_batch['global_normal'][1]

tensor([0.0463, 0.0196, 0.0980, 0.0000, 0.0140, 0.0476, 0.0289, 0.0063, 0.0000,
        0.0517, 0.0805, 0.0817, 0.1171, 0.0210, 0.0317, 0.0070, 0.0098, 0.0308,
        0.1851, 0.0000, 0.1431, 0.0000, 0.0789, 0.0344, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0102, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0135, 0.0000, 0.0115, 0.0300, 0.0024,
        0.0037, 0.0100, 0.0648])

In [7]:
val_batch['portion_normal'][1]

tensor([4.6256e-02, 1.9608e-02, 9.9990e+03, 0.0000e+00, 1.5402e-02, 4.7619e-02,
        2.8886e-02, 6.3006e-03, 0.0000e+00, 9.9990e+03, 8.0460e-02, 8.1696e-02,
        1.1712e-01, 9.9990e+03, 3.1746e-02, 9.9990e+03, 9.9990e+03, 9.9990e+03,
        9.9990e+03, 0.0000e+00, 1.4846e-01, 9.9990e+03, 9.9990e+03, 9.9990e+03,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 9.9990e+03, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 9.9990e+03, 9.9990e+03, 9.9990e+03,
        9.9990e+03, 9.9990e+03, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 9.9990e+03, 9.9990e+03, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 1.3536e-02, 0.0000e+00, 1.1454e-02, 2.9985e-02, 9.9990e+03,
        9.9990e+03, 4.8996e-02, 6.4836e-02])

In [5]:
val_batch['miss_matrix'][1]

tensor([1., 1., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 1., 0., 0., 0.,
        0., 1., 1., 0., 0., 0., 1., 1., 1., 1., 0., 1., 1., 1., 1., 0., 0., 0.,
        0., 0., 1., 1., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0.,
        0., 1., 1.])

In [3]:
train_dataloader = DataLoader(train_dataset, 
                             batch_size=10, 
                             collate_fn=train_dataset.collater, 
                             shuffle=False,
                             num_workers=2)
batch = None
for i in train_dataloader:
    batch = i
    break

In [4]:
batch['src_data']

tensor([[0.0000e+00, 6.4000e-01, 6.4000e-01, 0.0000e+00, 3.2000e-01, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 6.4000e-01,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 3.2000e-01, 0.0000e+00, 1.2900e+00,
         1.9300e+00, 0.0000e+00, 9.6000e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 7.7800e-01, 0.0000e+00, 0.0000e+00,
         3.7560e+00, 6.1000e+01, 2.7800e+02],
        [2.1000e-01, 2.8000e-01, 5.0000e-01, 0.0000e+00, 1.4000e-01, 2.8000e-01,
         2.1000e-01, 7.0000e-02, 0.0000e+00, 9.4000e-01, 2.1000e-01, 7.9000e-01,
         6.5000e-01, 2.1000e-01, 1.4000e-01, 1.4000e-01, 7.0000

In [ ]:
batch["global_normal"]

In [8]:
batch["portion_normal"][0]

tensor([1.0101e-02, 9.9990e+03, 2.0000e+00, 1.0000e+00, 5.1724e-02, 9.9990e+03,
        9.9990e+03, 1.0000e+00, 1.0000e+00, 0.0000e+00, 9.9990e+03, 1.4998e-01,
        6.9164e-02, 1.5603e-01, 1.6014e-01, 9.9990e+03, 1.1909e-01, 9.9990e+03,
        9.9990e+03, 0.0000e+00, 0.0000e+00, 9.9990e+03, 0.0000e+00])

In [ ]:
a = torch.randint(0,10, size=(5,4,3)).float()
print(a)
f = torch.nn.Softmax(dim=1)
f(a)


In [ ]:
a = torch.randint(0,10,size=(5,12,4)).float()
print(a)
model = torch.nn.Linear(4,10)
model(a[:,1,:]).shape

In [ ]:
a[:,0].reshape(-1,1)

In [ ]:
model(a[:,0].reshape(-1,1,1)).permute(0, 2, 1).shape

In [ ]:
isinstance(model, torch.nn.Embedding)

In [ ]:
a = torch.randint(0,2,size=(5, 3))
b = torch.nn.Parameter(torch.randn(1, 4))
print(b)
print(a)

miss_matric = a.unsqueeze (-1).expand(5,3,4)
print(miss_matric)
miss_matric*b

In [ ]:
a = torch.randint(0,10,size=(5,3)).float()
model = torch.nn.Conv1d(in_channels=1, out_channels=8, kernel_size=1, stride=1)
print(a)
embedding_out = torch.tensor([])
embedding_out = torch.cat((embedding_out, model(a[:,0].reshape(-1,1,1)).permute(0, 2, 1)), dim = 1)
embedding_out = torch.cat((embedding_out, model(a[:,1].reshape(-1,1,1)).permute(0, 2, 1)), dim = 1)
print(embedding_out.shape)
embedding_out

In [ ]:
b.shape

### 模型5精度测定

In [1]:
from dataModule.dataset1 import FlatDataset
from modelModule.model5 import VAE5
from torch.utils.data import DataLoader
import torch
import numpy as np
import pandas as pd
import pickle

In [2]:
## 模型参数载入
model = VAE5(dim=9, 
            pro_types=pickle.load(open('Breast\Breast_pro_type.pkl', 'rb')),
            replace_dict=pickle.load(open('Breast\Breast数值置换表.pkl', 'rb')))# nhead=3
# checkpoint_path = r'模型参数保存/Credit_Card_model5_norm_way_minmax_norm_/version_1/checkpoints/epoch=31-step=3008.ckpt'    
data_norm = 'minmax_norm'  # 数据集采用正则化方式minmax_norm,mean_norm

# checkpoint = torch.load(checkpoint_path)
# model.load_state_dict({k.replace('model.',''):v for k,v in checkpoint['state_dict'].items()}) 
# model.eval()

print('载入成功')

载入成功


In [3]:
miss_file = 'Breast\Breast模拟缺失_0_3.csv'      # 缺失数据集
miss_data = pd.read_csv(miss_file)

c, _ = model.inference(miss_data)


In [5]:
c.shape

(683, 9)

In [ ]:
# ## 模型参数载入
# model = VAE5(dim=23, pro_types=pickle.load(open('Credit_Card\Credit_Card_pro_type.pkl', 'rb'))) # nhead=3
# checkpoint_path = r'模型参数保存/Credit_Card_model5_norm_way_minmax_norm_/version_1/checkpoints/epoch=31-step=3008.ckpt'    
# data_norm = 'minmax_norm'  # 数据集采用正则化方式minmax_norm,mean_norm
# train_csv = r'Credit_Card_model_train.csv'
# test_csv = r'Credit_Card_model_test.csv'

# checkpoint = torch.load(checkpoint_path)
# model.load_state_dict({k.replace('model.',''):v for k,v in checkpoint['state_dict'].items()}) 
# model.eval()
# ## 载入训练集的均值(最小)方差(最大)
# train_dataset = FlatDataset(train_csv, data_norm=data_norm, pro_type_file='Credit_Card_pro_type.pkl')
# model.get_global_min_max(dataset=train_dataset)
# print('载入成功')

In [ ]:

# test_dataset = FlatDataset(test_csv, data_norm=data_norm, pro_type_file='Credit_Card_pro_type.pkl')
# data_size = 10
# src_data = np.stack([test_dataset[index]['src_data'] for index in range(data_size)], axis=0)
# normal_data = np.stack([test_dataset[index]['global_normal_data'] for index in range(data_size)], axis=0)
# portion_normal_data = np.stack([test_dataset[index]['portion_normal_data'] for index in range(data_size)], axis=0)
# _, miss_matrix = get_missing(src_data, 0.3) # 得到缺失矩阵
# input = portion_normal_data.copy()
# input[miss_matrix == 0] = np.nan

In [ ]:
# imputed_data, output = model.inference(input, miss_matrix) # inference输出的结果已经重构后的结果
# imputed_data = M_matrix * miss_data + (1-M_matrix) * imputed_data.data.numpy()

In [ ]:
# print(normal_data[0])
# print(output[0])


In [ ]:
imputed_data[1]

In [ ]:
result_show(src_data, imputed_data, miss_matrix)

In [ ]:
imputed_data

In [ ]:
imputed_data.shape

In [ ]:
# test_dataset = FlatDataset('Letter_test.csv', data_norm='mean_norm') # minmax_norm
# index = 102
# src_data, M_matrix, miss_data = test_dataset[index]['src_data'].reshape(1,-1), test_dataset[index]['global_miss_matrix'].reshape(1,-1), test_dataset[index]['global_miss_data'].reshape(1,-1)
# src_data = test_dataset[index]['src_data'].reshape(1,-1)
# normal_data = test_dataset[index]['global_normal_data'].reshape(1,-1)
# miss_data, M_matrix = get_missing(normal_data, 0.3) # 产生掩码数据，并采用随机数填补

# imputed_data, _, _ = model(torch.from_numpy(miss_data).float(), torch.from_numpy(M_matrix).float()) 
# imputed_data = M_matrix * miss_data + (1-M_matrix) * imputed_data.data.numpy() # 得到最终插补结果,数据由[0,1]范围内
# res_imputed_data = np.around(restore_data(imputed_data, test_dataset.Max_Val, test_dataset.Min_Val))
# result_show(src_data, res_imputed_data, M_matrix)

In [ ]:
# batch测试
# src_data, miss_data, M_matrix = batch['src_data'], batch['miss_data'], batch['miss_matrix']
# normal_data = batch['normal_data']
# imputed_data, _, _ = model(miss_data, M_matrix)
# imputed_data = M_matrix * miss_data + (1-M_matrix) * imputed_data.data.numpy()
# res_imputed_data = np.around(restore_data(imputed_data, dataset.Max_Val, dataset.Min_Val))